In [3]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import psycopg2
from datetime import datetime
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

# Define your RDS instance connection parameters
host = "projects-database.cxgcu68ksihx.us-east-1.rds.amazonaws.com"          # RDS endpoint, e.g., <your-db>.rds.amazonaws.com
port = "5432"                       # Default port for PostgreSQL
dbname = "postgres"               # Database name
user = "postgres"                   # Username
password = "DATA602_project"        # Password
try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=dbname,
        user=user,
        password=password
    )

    cursor = connection.cursor()

    cursor.execute("SELECT * FROM sales_data;")

    results = cursor.fetchall()

    colnames = [desc[0] for desc in cursor.description]

    # Create a Pandas DataFrame
    ret = pd.DataFrame(results, columns=colnames)

    #for row in results:
        #print(row)

except Exception as error:
    print(f"Error connecting to PostgreSQL: {error}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
        results
ret.head()

def frozenset_to_string(fset):
    if isinstance(fset, frozenset):
        return ', '.join(str(item) for item in fset)  # Convert each item to string
    elif isinstance(fset, (set, list)):
        return ', '.join(str(item) for item in fset)
    else:
        return str(fset)

def frozenset_to_string(fset):
    if isinstance(fset, frozenset):
        return ', '.join(map(str, fset))  # Convert each item in frozenset to string
    return fset

# Helper function to retrieve MRP values for a list of Product IDs
def get_mrp_values(product_ids, ret_df):
    # Check if product_ids is a frozenset, list, or other iterable
    if isinstance(product_ids, frozenset):
        product_ids_list = list(product_ids)  # Convert to list
    else:
        product_ids_list = [int(pid.strip()) for pid in product_ids.split(',')]

    # Fetch MRP values for the provided product IDs
    mrp_values = ret_df[ret_df['description'].isin(product_ids_list)]['unitprice'].tolist()
    return mrp_values


# Function to filter rows where antecedent MRP is less than consequent MRP
def filter_up_sell_candidates(up_sell_df, ret_df):
    valid_rows = []

    for _, row in up_sell_df.iterrows():
        # Get the product IDs for antecedents and consequents
        antecedents = row['antecedents']
        consequents = row['consequents']

        # Fetch MRP values for both antecedents and consequents
        antecedent_mrps = get_mrp_values(antecedents, ret_df)
        consequent_mrps = get_mrp_values(consequents, ret_df)

        # Check if all antecedent MRP values are less than all consequent MRP values
        if max(antecedent_mrps) < min(consequent_mrps):
            valid_rows.append(row)

    # Return the valid rows as a new dataframe
    return pd.DataFrame(valid_rows)

all_cross_sell_results = []
all_up_sell_results = []

# Loop over each unique Store_ID
for country in ret['country'].unique():
    # Filter data for the current Store_ID
    country_data = ret[ret['country'] == country]

    # Prepare the basket as before
    basket = (
        country_data.groupby(['invoiceno', 'description'])['quantity'].sum()
        .unstack()
        .reset_index()
        .fillna(0)
        .set_index('invoiceno')
    )

    # Encode the basket for frequent pattern mining
    my_basket_sets = basket.map(lambda x: 1 if x > 0 else 0)

    # Generate frequent itemsets and association rules
    my_frequent_itemsets = fpgrowth(my_basket_sets, min_support=0.1, use_colnames=True)
    num_itemsets = my_frequent_itemsets.shape[0]  # Number of rows in frequent itemsets dataframe
    my_rules = association_rules(my_frequent_itemsets, metric="lift", min_threshold=0.001,num_itemsets=num_itemsets)

    # Sort cross-sell rules by lift and confidence
    my_cross_sell_rules = my_rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])

    # Convert frozensets to strings
    my_cross_sell_rules['antecedents'] = my_cross_sell_rules['antecedents'].apply(frozenset_to_string)
    my_cross_sell_rules['consequents'] = my_cross_sell_rules['consequents'].apply(frozenset_to_string)

    # Add Store_ID to the cross-sell rules
    my_cross_sell_rules['country'] = country

    # Append cross-sell rules for this store to the list
    all_cross_sell_results.append(my_cross_sell_rules)

    # # Merge store data with product categories
    # ups = pd.merge(
    #     country_data[['description', 'unitprice']],
    #     cat[['description', 'Category']],
    #     on='description'
    # )

    # Create a dictionary for fast category lookups
    # product_category_map = dict(zip(ups['description'], ups['Category']))

#     # Add category information to the rules
#     my_rules['antecedent_category'] = my_rules['antecedents'].apply(lambda x: get_category(list(x)[0], product_category_map))
#     my_rules['consequent_category'] = my_rules['consequents'].apply(lambda x: get_category(list(x)[0], product_category_map))

#     # Filter up-sell candidates with same category
#     up_sell_candidates = my_rules[my_rules['antecedent_category'] == my_rules['consequent_category']]
#     up_sell_candidates_sorted = up_sell_candidates.sort_values(by=['confidence', 'lift'], ascending=[False, False])

#     # Filter up-sell candidates where antecedent MRP < consequent MRP
#     filtered_up_sell_candidates = filter_up_sell_candidates(up_sell_candidates_sorted, store_data)

#     # Add Store_ID to the up-sell candidates
#     filtered_up_sell_candidates['invoiceno'] = country

#     # Append up-sell candidates for this store to the list
#     all_up_sell_results.append(filtered_up_sell_candidates)

# # Concatenate all cross-sell and up-sell results into their respective dataframes
final_cross_sell_results = pd.concat(all_cross_sell_results, ignore_index=True)
# final_up_sell_results = pd.concat(all_up_sell_results, ignore_index=True)

# Display the final results for both cross-sell and up-sell
print("Final Cross-Sell Rules with Store_ID Labels:")
print(final_cross_sell_results[['country', 'antecedents', 'consequents', 'confidence', 'lift']].head())


# print("\nFinal Up-Sell Candidates with Store_ID Labels:")
# print(final_up_sell_results[['country', 'antecedents', 'consequents', 'confidence', 'lift']].head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PostgreSQL connection is closed


/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The input DataFrame `df` containing the frequent itemsets is empty.

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import psycopg2
from datetime import datetime
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

# Define your RDS instance connection parameters
host = "projects-database.cxgcu68ksihx.us-east-1.rds.amazonaws.com"          # RDS endpoint, e.g., <your-db>.rds.amazonaws.com
port = "5432"                       # Default port for PostgreSQL
dbname = "postgres"               # Database name
user = "postgres"                   # Username
password = "DATA602_project"        # Password
try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=dbname,
        user=user,
        password=password
    )

    cursor = connection.cursor()

    cursor.execute("SELECT * FROM sales_data;")

    results = cursor.fetchall()

    colnames = [desc[0] for desc in cursor.description]

    # Create a Pandas DataFrame
    ret = pd.DataFrame(results, columns=colnames)

    #for row in results:
        #print(row)

except Exception as error:
    print(f"Error connecting to PostgreSQL: {error}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
        results
ret.head()

def frozenset_to_string(fset):
    if isinstance(fset, frozenset):
        return ', '.join(str(item) for item in fset)  # Convert each item to string
    elif isinstance(fset, (set, list)):
        return ', '.join(str(item) for item in fset)
    else:
        return str(fset)

def frozenset_to_string(fset):
    if isinstance(fset, frozenset):
        return ', '.join(map(str, fset))  # Convert each item in frozenset to string
    return fset

# Helper function to retrieve MRP values for a list of Product IDs
def get_mrp_values(product_ids, ret_df):
    # Check if product_ids is a frozenset, list, or other iterable
    if isinstance(product_ids, frozenset):
        product_ids_list = list(product_ids)  # Convert to list
    else:
        product_ids_list = [int(pid.strip()) for pid in product_ids.split(',')]

    # Fetch MRP values for the provided product IDs
    mrp_values = ret_df[ret_df['description'].isin(product_ids_list)]['unitprice'].tolist()
    return mrp_values


# Function to filter rows where antecedent MRP is less than consequent MRP
def filter_up_sell_candidates(up_sell_df, ret_df):
    valid_rows = []

    for _, row in up_sell_df.iterrows():
        # Get the product IDs for antecedents and consequents
        antecedents = row['antecedents']
        consequents = row['consequents']

        # Fetch MRP values for both antecedents and consequents
        antecedent_mrps = get_mrp_values(antecedents, ret_df)
        consequent_mrps = get_mrp_values(consequents, ret_df)

        # Check if all antecedent MRP values are less than all consequent MRP values
        if max(antecedent_mrps) < min(consequent_mrps):
            valid_rows.append(row)

    # Return the valid rows as a new dataframe
    return pd.DataFrame(valid_rows)

all_cross_sell_results = []
all_up_sell_results = []

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PostgreSQL connection is closed


In [5]:
ret

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,category,column1
0,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Lantern,1
1,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,Coat Hanger,2
2,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Water Bottle,3
3,5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,Nesting Box,5
4,7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,Hand Warmer,7
...,...,...,...,...,...,...,...,...,...,...,...
281099,521720,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680,France,Cutlery,521720
281100,521721,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,Napkins,521721
281101,521723,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,Cutlery,521723
281102,521724,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,Cutlery Set,521724


In [36]:
def get_category(item, product_category_map):
    # Fetch category from the product_category_map
    return product_category_map.get(item, None)
# Loop over each unique Store_ID
for country in ret['country'].unique():
    # Filter data for the current Store_ID
    country_data = ret[ret['country'] == country]

    # Prepare the basket as before
    basket = (
        country_data.groupby(['invoiceno', 'description'])['quantity'].sum()
        .unstack()
        .reset_index()
        .fillna(0)
        .set_index('invoiceno')
    )

    # Encode the basket for frequent pattern mining
    my_basket_sets = basket.map(lambda x: 1 if x > 0 else 0)

    # Generate frequent itemsets and association rules
    my_frequent_itemsets = fpgrowth(my_basket_sets, min_support=0.03, use_colnames=True)
    num_itemsets = my_frequent_itemsets.shape[0]  # Number of rows in frequent itemsets dataframe
    my_rules = association_rules(my_frequent_itemsets, metric="lift", min_threshold=0.01,num_itemsets=num_itemsets)

    # Sort cross-sell rules by lift and confidence
    my_cross_sell_rules = my_rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])

    # Convert frozensets to strings
    my_cross_sell_rules['antecedents'] = my_cross_sell_rules['antecedents'].apply(frozenset_to_string)
    my_cross_sell_rules['consequents'] = my_cross_sell_rules['consequents'].apply(frozenset_to_string)

    # Add Store_ID to the cross-sell rules
    my_cross_sell_rules['country'] = country

    # Append cross-sell rules for this store to the list
    all_cross_sell_results.append(my_cross_sell_rules)

    # Merge store data with product categories
    ups = pd.merge(
        country_data[['description', 'unitprice']],
        ret[['description', 'category']],
        on='description'
    )

    # Create a dictionary for fast category lookups
    product_category_map = dict(zip(ups['description'], ups['category']))

    # Add category information to the rules
    my_rules['antecedent_category'] = my_rules['antecedents'].apply(lambda x: get_category(list(x)[0], product_category_map))
    my_rules['consequent_category'] = my_rules['consequents'].apply(lambda x: get_category(list(x)[0], product_category_map))

    # Filter up-sell candidates with same category
    up_sell_candidates = my_rules[my_rules['antecedent_category'] == my_rules['consequent_category']]
    up_sell_candidates_sorted = up_sell_candidates.sort_values(by=['confidence', 'lift'], ascending=[False, False])

    # Filter up-sell candidates where antecedent MRP < consequent MRP
    filtered_up_sell_candidates = filter_up_sell_candidates(up_sell_candidates_sorted, country_data)

    # Add Store_ID to the up-sell candidates
    filtered_up_sell_candidates['country'] = country
    # Append up-sell candidates for this satore to the list
    all_up_sell_results.append(filtered_up_sell_candidates)

# Concatenate all cross-sell and up-sell results into their respective dataframes
final_cross_sell_results = pd.concat(all_cross_sell_results, ignore_index=True)
final_up_sell_results = pd.concat(all_up_sell_results, ignore_index=True)

# Display the final results for both cross-sell and up-sell
print("Final Cross-Sell Rules with Country Labels:")
print(final_cross_sell_results[['country', 'antecedents','support', 'consequents', 'confidence', 'lift']])


print("\nFinal Up-Sell Candidates with Country Labels:")
print(final_up_sell_results[['country', 'antecedents', 'consequents', 'confidence', 'lift']].head())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py

Final Cross-Sell Rules with Country Labels:
              country                        antecedents   support  \
0      United Kingdom         WOODEN FRAME ANTIQUE WHITE  0.028440   
1      United Kingdom  WOODEN PICTURE FRAME WHITE FINISH  0.028440   
2      United Kingdom            JUMBO BAG RED RETROSPOT  0.021804   
3      United Kingdom            LUNCH BAG RED RETROSPOT  0.021804   
4      United Kingdom            JUMBO BAG RED RETROSPOT  0.028575   
...               ...                                ...       ...   
10931            EIRE         SET OF 3 REGENCY CAKE TINS  0.038793   
10932            EIRE    ROSES REGENCY TEACUP AND SAUCER  0.043103   
10933            EIRE         SET OF 3 REGENCY CAKE TINS  0.043103   
10934            EIRE   SET OF 3 CAKE TINS PANTRY DESIGN  0.030172   
10935            EIRE    ROSES REGENCY TEACUP AND SAUCER  0.030172   

                             consequents  confidence       lift  
0      WOODEN PICTURE FRAME WHITE FINISH    0.582

<ipython-input-36-60d41576d5e8>:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_cross_sell_results = pd.concat(all_cross_sell_results, ignore_index=True)


##  Below code is just for backup, delete this before submitting it as a Final Project file

In [16]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Function to dynamically adjust min_support
def find_min_support(country_data, initial_support=0.05, step=0.01, metric="lift", min_threshold=0.01):
    min_support = initial_support  # Initialize min_support
    while min_support > 0:  # Prevent infinite loop
        # Prepare the basket
        basket = (
            country_data.groupby(['invoiceno', 'description'])['quantity'].sum()
            .unstack()
            .reset_index()
            .fillna(0)
            .set_index('invoiceno')
        )

        # Encode the basket for frequent pattern mining
        my_basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

        # Generate frequent itemsets
        my_frequent_itemsets = fpgrowth(my_basket_sets, min_support=min_support, use_colnames=True)

        # If no itemsets are found, reduce min_support further
        if my_frequent_itemsets.empty:
            min_support -= step
            continue

        # Generate association rules
        num_itemsets = my_frequent_itemsets.shape[0]
        my_rules = association_rules(my_frequent_itemsets, metric=metric, min_threshold=min_threshold, num_itemsets=num_itemsets)

        # Check if all antecedents and consequents are single products
        if my_rules['antecedents'].apply(len).max() == 1 and my_rules['consequents'].apply(len).max() == 1:
            return my_rules, min_support  # Return the rules and the min_support used

        # Reduce min_support and try again
        min_support -= step

    return pd.DataFrame(), min_support  # Return empty DataFrame if no suitable rules are found

# Loop over each unique country
all_cross_sell_results = []

for country in ret['country'].unique():
    # Filter data for the current country
    country_data = ret[ret['country'] == country]

    # Dynamically find the optimal min_support
    my_cross_sell_rules, final_min_support = find_min_support(country_data)

    if not my_cross_sell_rules.empty:
        # Convert frozensets to strings
        my_cross_sell_rules['antecedents'] = my_cross_sell_rules['antecedents'].apply(frozenset_to_string)
        my_cross_sell_rules['consequents'] = my_cross_sell_rules['consequents'].apply(frozenset_to_string)

        # Add country label to the cross-sell rules
        my_cross_sell_rules['country'] = country

        # Append the results
        all_cross_sell_results.append(my_cross_sell_rules)

# Concatenate all results into a single DataFrame
final_cross_sell_results = pd.concat(all_cross_sell_results, ignore_index=True)

# Display the final results
print("Final Cross-Sell Rules with Single Products in Antecedents and Consequents:")
print(final_cross_sell_results[['country', 'antecedents', 'support', 'consequents', 'confidence', 'lift']])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-16-a7c9ad7eace0>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
<ipython-input-16-a7c9ad7eace0>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets = basket.applymap(lambda x: 1 if x > 0 else 

Final Cross-Sell Rules with Single Products in Antecedents and Consequents:
           country                         antecedents   support  \
0   United Kingdom          WOODEN FRAME ANTIQUE WHITE  0.028440   
1   United Kingdom   WOODEN PICTURE FRAME WHITE FINISH  0.028440   
2   United Kingdom             JUMBO BAG RED RETROSPOT  0.021804   
3   United Kingdom             LUNCH BAG RED RETROSPOT  0.021804   
4   United Kingdom             JUMBO BAG RED RETROSPOT  0.028575   
..             ...                                 ...       ...   
67         Germany              WOODLAND CHARLOTTE BAG  0.059242   
68         Germany  ROUND SNACK BOXES SET OF4 WOODLAND  0.061611   
69         Germany                  SPACEBOY LUNCH BOX  0.061611   
70         Germany            PLASTERS IN TIN SPACEBOY  0.052133   
71         Germany    PLASTERS IN TIN WOODLAND ANIMALS  0.052133   

                           consequents  confidence       lift  
0    WOODEN PICTURE FRAME WHITE FINISH    0